## Importing the Libraries

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import warnings

## Loading and Exploring the Dataset

#### 1. Loading the Dataset

In [3]:
try:
    df = pd.read_csv("Online Retail.csv", encoding='latin1')
    print("Data loaded successfully.")
except FileNotFoundError:
    print("Error: The file 'Online Retail.csv' was not found.")
    exit()

Data loaded successfully.


#### 2. Exploring the Dataset

In [4]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [6]:
df.info()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [8]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,541909.0,9.552250,218.081158,-80995.00,1.00,3.00,10.00,80995.0
UnitPrice,541909.0,4.611114,96.759853,-11062.06,1.25,2.08,4.13,38970.0
CustomerID,406829.0,15287.690570,1713.600303,12346.00,13953.00,15152.00,16791.00,18287.0


The negative values of minimum for Quantity and UnitPrice is due to the presence of orders that were cancelled.

## Data Cleaning and Preprocessing

To prepare the data for customer segmentation, specific steps are taken to handle incomplete or irrelevant records. First, rows where the `CustomerID` is missing are dropped, as customer identification is essential for the analysis. Without a unique ID, customers cannot be segmented, making these records unusable for the intended purpose.

Second, any cancelled orders are removed from the dataset. These are identified by an `InvoiceNo` that contains the letter 'C'. In a retail context, an invoice number containing 'C' often signifies a credit note or a cancelled order. Also, from the descriptive statistics (count, mean, std, min, max, and quartiles), it is observed that the minimum values for `Quantity` and `UnitPrice` are negative, which is a direct consequence of the cancelled orders. Removing these records prevents the analysis of customer frequency and monetary value from being skewed by invalid purchases.

Third, the `InvoiceDate` column is converted to a datetime data type to enable time-based calculations, such as determining the "Recency" of a customer's most recent purchase.

In [30]:
df.dropna(subset=['CustomerID'], inplace=True)
df = df[~df['InvoiceNo'].str.contains('C', na=False)]

The **`InvoiceDay`** column is added to the data so that each invoice's date can be analyzed on a daily basis. This is done to enable versatile time-based analyses, as the data can then be analyzed by individual days or grouped into different daily intervals.

In [34]:
df['InvoiceDay'] = df['InvoiceDate'].apply(lambda x: dt.datetime(x.year, x.month, x.day))

A new variable, `TotalPrice`, is created to represent the total cost of each product. This is calculated by multiplying the quantity of each item by its unit price. Based on this, the total expenses for each customer can then be computed and used for financial analysis.

In [35]:
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

In [37]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice,InvoiceDay
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,2010-12-01
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010-12-01
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,2010-12-01
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010-12-01
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010-12-01


In [38]:
df.shape

(397924, 10)

In [39]:
df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
TotalPrice     0
InvoiceDay     0
dtype: int64

## Feature Engineering: Building the RFM Model

The RFM model is constructed to summarize customer purchasing behavior into three core metrics: **Recency**, **Frequency**, and **Monetary** value. These metrics are used as features for a clustering algorithm.
- **Recency** is a measure of how recently a customer's last purchase was made. A lower value indicates a more recent purchase, which is considered a better score.
- **Frequency** is the measure of how often purchases are made by a customer. A higher value for this metric is considered more favorable.
- **Monetary** value represents the total amount of money a customer has spent. A higher value is preferred, as it signifies a greater financial contribution.

Here:
* A **snapshot date** is created by adding one day to the most recent invoice date. This serves as the reference point for calculating recency.
* The data is **grouped by `CustomerID`** to perform calculations for each individual customer.
* **Recency** is calculated by finding the difference in days between the snapshot date and the customer's last purchase date.
* **Frequency** is determined by counting the number of unique invoices for each customer.
* **Monetary** value is calculated by summing the total price of all purchases for each customer.
* The columns are **renamed** to 'Recency', 'Frequency', and 'Monetary' for clarity.
* Finally, the `CustomerID` is **reset from the index** to become a regular column in the new `rfm_df` DataFrame.

In [42]:
snapshot_date = df['InvoiceDay'].max() + dt.timedelta(days=1)
rfm_df = df.groupby('CustomerID').agg({
    'InvoiceDay': lambda date: (snapshot_date - date.max()).days,
    'InvoiceNo': lambda num: len(num.unique()),
    'TotalPrice': lambda price: price.sum()
})
rfm_df.columns = ['Recency', 'Frequency', 'Monetary']
rfm_df = rfm_df.reset_index()

In [43]:
rfm_df.head()

,CustomerID,Recency,Frequency,Monetary
0,12346.0,326,1,77183.60
1,12347.0,3,7,4310.00
2,12348.0,76,4,1797.24
3,12349.0,19,1,1757.55
4,12350.0,311,1,334.40
